In [1]:
from optparse import OptionParser
import numpy as nm

import sys
sys.path.append('.')

from sfepy.base.base import IndexedStruct
from sfepy.discrete import (FieldVariable, Material, Integral, Function,
                            Equation, Equations, Problem)
from sfepy.discrete.fem import Mesh, FEDomain, Field
from sfepy.terms import Term
from sfepy.discrete.conditions import Conditions, EssentialBC
from sfepy.solvers.ls import ScipyDirect
from sfepy.solvers.nls import Newton

In [2]:
def shift_u_fun(ts, coors, bc=None, problem=None, shift=0.0):
    """
    Define a displacement depending on the y coordinate.
    """
    val = shift * coors[:,1]**2

    return val


In [3]:
dims = nm.array((1.0,1.0,1.0))
shape = nm.array((20,20,20))

In [4]:
centre = nm.array((0.0,0.0,0.0))

In [5]:
E_f, nu_f, E_m, nu_m  = 160.e9, 0.2, 5.e9, 0.3

In [6]:
from sfepy.mesh import mesh_generators
from sfepy.terms.terms_elastic import CauchyStressTerm
from sfepy.mechanics.matcoefs import stiffness_from_lame,stiffness_from_youngpoisson
from sfepy.mechanics.tensors import get_von_mises_stress
from sfepy.base.base import Struct
from sfepy.discrete.probes import LineProbe
from its2D_2 import stress_strain
from its2D_3 import nodal_stress
from sfepy.discrete.projections import project_by_component
import uuid
import os
from pyamg import smoothed_aggregation_solver
from scipy.sparse.linalg import bicgstab, spilu


In [7]:
def get_mat1(ts,coors, mode=None,**kwargs):
    if mode == 'qp':
        #cnf = pb.conf
        # get material coefficients
        
            # given values
            #E_f, nu_f, E_m, nu_m  = 160.e9, 0.2, 5.e9, 0.4

        nqp = coors.shape[0]
        #nel = pb.domain.mesh.n_el
        #nqpe = nqp / nel
        out = nm.zeros((nqp, 6, 6), dtype=nm.float64)
        D_m = stiffness_from_youngpoisson(3, E_m, nu_m)
        D_f = stiffness_from_youngpoisson(3, E_f, nu_f)
        out[0:nqp, ...] = D_f
        for i in nm.arange(nqp):
            if coors[i,2]>-0.1 and coors[i,2]<0.1:
                out[i,...] = D_m
        return {'D': out}


In [8]:
mesh = mesh_generators.gen_block_mesh(dims,shape,centre,name='block')
domain = FEDomain('domain', mesh)

min_x, max_x = domain.get_mesh_bounding_box()[:,0]
min_y, max_y = domain.get_mesh_bounding_box()[:,1]
min_z, max_z = domain.get_mesh_bounding_box()[:,2]

sfepy: generating 8000 vertices...
sfepy: ...done
sfepy: generating 6859 cells...
sfepy: ...done


In [9]:
R1 = domain.create_region('Ym',
                                  'vertices in z < %.10f' % (max_z/2))


In [10]:
R2 = domain.create_region('Yf',
                                  'vertices in z >= %.10f' % (min_z/2))

In [11]:
omega = domain.create_region('Omega', 'all')

In [12]:
eps = 1e-8 * (max_x - min_x)

In [13]:
gamma1 = domain.create_region('Left',
                                  'vertices in x < %.10f' % (min_x + eps), 
                                  'facet')

In [14]:
gamma2 = domain.create_region('Right',
                                  'vertices in x > %.10f' % (max_x - eps),
                                  'facet')

In [15]:
gamma3 = domain.create_region('Front',
                                  'vertices in y < %.10f' % (min_y + eps),
                                  'facet')

In [165]:
gamma4 = domain.create_region('Back',
                                  'vertices in y > %.10f' % (max_y - eps),
                                  'facet')

In [166]:
gamma5 = domain.create_region('Bottom',
                                  'vertices in z < %.10f' % (min_z + eps),
                                  'facet')

In [167]:
gamma6 = domain.create_region('Top',
                                  'vertices in z > %.10f' % (max_z - eps),
                                  'facet')

In [168]:
field = Field.from_args('fu', nm.float64, 'vector', omega, approx_order=2)


In [169]:
u = FieldVariable('u', 'unknown', field)

In [170]:
v = FieldVariable('v', 'test', field, primary_var_name='u')

In [171]:
get_mat = Function('get_mat1',get_mat1)

In [172]:
integral = Integral('i', order=3)

In [173]:
mu=1.1

In [174]:
lam=1.0


In [175]:
D = stiffness_from_lame(3,lam, mu)

In [176]:
mat = Material('Mat', D=D)

In [177]:
t1 = Term.new('dw_lin_elastic(Mat.D, v, u)',
         integral, omega, Mat=mat, v=v, u=u)


In [178]:
f = Material('f', val=[[0.0], [0.0],[0.0]])

In [179]:
t2 = Term.new('dw_volume_lvf(f.val, v)', integral, omega, f=f, v=v)

In [180]:
eq = Equation('balance', t1 + t2)

In [181]:
eqs = Equations([eq])

In [182]:
fix_u = EssentialBC('fix_u', gamma1, {'u.all' : 0.0})

In [183]:
left_bc  = EssentialBC('Left',  gamma1, {'u.0' : 0.0})

In [184]:
right_bc = EssentialBC('Right', gamma2, {'u.0' : 0.0})

In [185]:
back_bc = EssentialBC('Front', gamma3, {'u.1' : 0.0})

In [186]:
bottom_bc = EssentialBC('Bottom', gamma5, {'u.all' : 0.0})

In [187]:
top_bc = EssentialBC('Top', gamma6, {'u.2' : 0.2})

In [188]:
front_bc = EssentialBC('Back', gamma4, {'u.1' : 0.0})

In [189]:
bc=[left_bc,right_bc,back_bc,front_bc,bottom_bc,top_bc]

In [190]:
conf = Struct(method='bcgsl', precond='jacobi', sub_precond=None,
                  i_max=10000, eps_a=1e-50, eps_r=1e-10, eps_d=1e4,
                  verbose=True)

In [191]:
from sfepy.solvers.ls import ScipyDirect,ScipyIterative,PyAMGSolver,PETScKrylovSolver

In [192]:
ls = PETScKrylovSolver(conf)

In [193]:
nls_status = IndexedStruct()

In [194]:
nls = Newton({'i_max':1,'eps_a':1e-10}, lin_solver=ls, status=nls_status)

In [195]:
pb = Problem('elasticity', equations=eqs, nls=nls, ls=ls)

In [196]:
dd=pb.get_materials()['Mat']

In [197]:
dd.set_function(get_mat1)

In [198]:
pb.save_regions_as_groups('regions')

sfepy: saving regions as groups...
sfepy:   Omega
sfepy:   Ym
sfepy:   Yf
sfepy:   Left
sfepy:   Right
sfepy:   Front
sfepy:   Back
sfepy:   Bottom
sfepy:   Top
sfepy: ...done


In [199]:
pb.time_update(ebcs=Conditions(bc))

sfepy: updating variables...
sfepy: ...done
sfepy: setting up dof connectivities...
sfepy: ...done in 0.01 s
sfepy: matrix shape: (165945, 165945)
sfepy: assembling matrix graph...
sfepy: ...done in 3.34 s
sfepy: matrix structural nonzeros: 29264715 (1.06e-03% fill)


In [200]:
vec = pb.solve()

sfepy: updating materials...
sfepy:     Mat
sfepy:     f
sfepy: ...done in 0.05 s
sfepy: nls: iter: 0, residual: 5.280543e+10 (rel: 1.000000e+00)
sfepy: bcgsl(jacobi, None/proc) convergence: 2 (CONVERGED_RTOL)
sfepy:   rezidual:    0.10 [s]
sfepy:      solve:    7.98 [s]
sfepy:     matrix:    3.59 [s]
sfepy: warning: linear system solution precision is lower
sfepy: then the value set in solver options! (err = 1.548756e+00 < 1.000000e-10)
sfepy: nls: iter: 1, residual: 1.548755e+00 (rel: 2.932947e-11)


In [141]:
pb.

{'active_only': True,
 'conf': Struct,
 'domain': FEDomain:domain,
 'ebcs': Conditions,
 'epbcs': Conditions,
 'equations': Equations,
 'evaluator': BasicEvaluator,
 'fields': {'fu': H1NodalVolumeField:fu},
 'file_per_var': False,
 'float_format': None,
 'functions': None,
 'graph_changed': False,
 'ics': None,
 'integrals': None,
 'lcbcs': Conditions,
 'linearization': Struct,
 'matrix_hook': None,
 'mtx_a': <165945x165945 sparse matrix of type '<type 'numpy.float64'>'
 	with 29264715 stored elements in Compressed Sparse Row format>,
 'name': 'elasticity',
 'nls': Newton:nls.newton,
 'nls_iter_hook': None,
 'nls_status': IndexedStruct,
 'ofn_trunk': 'domain',
 'output_dir': '.',
 'output_format': 'vtk',
 'output_modes': {'h5': 'single', 'vtk': 'sequence'},
 'ts': TimeStepper}

In [143]:
lss=ScipyIterative(conf)
nls = Newton({'i_max':1,'eps_a':1e-10}, lin_solver=lss, status=nls_status)

sfepy: scipy solver bcgsl does not exist!
sfepy: using cg instead


In [144]:
pbs = Problem('elasticity', equations=eqs, nls=nls, ls=lss)

In [145]:
vec2 = pbs.solve()

sfepy: updating materials...
sfepy:     Mat
sfepy:     f
sfepy: ...done in 0.00 s
sfepy: nls: iter: 0, residual: 5.280543e+10 (rel: 1.000000e+00)


AttributeError: 'NoneType' object has no attribute 'data'